# Flag Detector

## Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_files
from kerastuner import HyperModel
from keras_tuner.tuners import Hyperband

/var/folders/5s/drh5kr3d4wj7hkt180wf48mc0000gn/T/ipykernel_4501/920603148.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


## Getting class names

In [2]:
files = load_files('./dataset/training_set', load_content= False)
total_countries = len(files.target_names)
print(files.target_names)

['Argentina', 'Belgium', 'Brazil', 'Columbia', 'Croatia', 'Denmark', 'England', 'France', 'Japan', 'Mexico', 'Portugal', 'Russia', 'Spain', 'Sweden', 'Switzerland', 'Uruguay']


## Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 classes = files.target_names,
                                                 batch_size = 32,
                                                 class_mode="categorical")

Found 587 images belonging to 16 classes.


## Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            classes = files.target_names,
                                            batch_size = 32,
                                            class_mode="categorical")

Found 65 images belonging to 16 classes.


## Hyper Model Class

In [5]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        cnn = tf.keras.models.Sequential()

        cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=self.input_shape))
        cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
        
        cnn.add(tf.keras.layers.Conv2D(
            filters=hp.Choice(
                'num_filters',
                values=[32, 64],
                default=64,
            ),
            kernel_size=3, 
            activation='relu'))
        cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
        
        cnn.add(tf.keras.layers.Flatten())
        
        cnn.add(tf.keras.layers.Dense(
            units=hp.Int(
                'units',
                min_value=32,
                max_value=512,
                step=32,
                default=128
            ),
            activation=hp.Choice(
                'dense_activation',
                values=['relu', 'tanh', 'sigmoid'],
                default='relu'
            )
        ))
        
        cnn.add(tf.keras.layers.Dense(
            units=self.num_classes,
            activation=hp.Choice(
                'dense_output_activation',
                values=['softmax', 'sigmoid'],
                default='sigmoid'
            )
        ))
        
        cnn.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
        
        return cnn

In [6]:
hypermodel = CNNHyperModel(input_shape=[64, 64, 3], num_classes=total_countries)

## Hyperparameter tuning the model

In [7]:
tuner = Hyperband(
    hypermodel,
    max_epochs=25,
    objective='val_accuracy',
    seed=1,
    executions_per_trial=2,
    directory='hyperband',
    project_name='flag_detector1'
)

INFO:tensorflow:Reloading Oracle from existing project hyperband/flag_detector1/oracle.json
INFO:tensorflow:Reloading Tuner from hyperband/flag_detector1/tuner0.json


In [8]:
tuner.search(x=training_set, validation_data=test_set, epochs=25)

INFO:tensorflow:Oracle triggered exit


In [9]:
# Show a summary of the search
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Results summary
Results in hyperband/flag_detector1
Showing 10 best trials
Trial summary
Hyperparameters:
num_filters: 64
units: 96
dense_activation: tanh
dense_output_activation: sigmoid
tuner/epochs: 25
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9230769276618958
Trial summary
Hyperparameters:
num_filters: 64
units: 448
dense_activation: tanh
dense_output_activation: sigmoid
tuner/epochs: 25
tuner/initial_epoch: 9
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.9230769276618958
Trial summary
Hyperparameters:
num_filters: 64
units: 288
dense_activation: sigmoid
dense_output_activation: sigmoid
tuner/epochs: 25
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9230769276618958
Trial summary
Hyperparameters:
num_filters: 32
units: 416
dense_activation: relu
dense_output_activation: sigmoid
tuner/epochs: 25
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9230769276618958
Trial summary
Hyperparameters:
num_filters: 64
units: 

### Evaluate best model

In [12]:
# Evaluate the best model.
loss, _ = best_model.evaluate(x = training_set)

print("\n Mean Squared Error:" + str(loss))

19/19 [==============================] - 0s 25ms/step - loss: 0.0068 - accuracy: 0.9625

 Mean Squared Error:0.006767168641090393


### Export best model

In [11]:
best_model.save('flag_detector.h5')